Reference
- "Numerical Methods for Engineers" Steven C. Chapra, and Raymond P. Canale

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_rows", None)

from scipy import linalg

### Problem 10.6
Use LU decomposition to determine the matrix inverse for the same system as in Prob.10.2.  Do not use a pivoting strategy, and check your results by verifying that $AA^{-1}=I$

$7x_1 + 2x_2 - 3x_3 = -12$

$2x_1 + 5x_2 - 3x_3 = -20$

$x_1 - x_2 - 6x_3 = -26$

### Answer

1. Define Function

In [2]:
def matrix_inverse(a, n):
    """
    This function implements matrix inversion with LU decomposition
      using the pseudocode from chapter 10 in the textbook
      
    This method doesn't use a pivoting strategy, no permutation due to zero pivot
    """
    # decompose the matrix
    decompose(a, n)
    
    # implements matrix inversion
    ai = [([0] * n) for i in range(n)] # for the inverse matrix, notice how to create it (avoid reference)
    for i in range(n):
        # for each column in the identity matrix
        # get the column of the identity matrix  at the right-hand-side
        b = [0] * n
        for j in range(n):
            if i == j:
                b[j] = 1
            else:
                b[j] = 0
        # calculate the corresponding column at the left-hand-side
        x = [0] * n
        substitute(a, n, b, x)
        for j in range(n):
            ai[j][i] = x[j]
                
    return ai
    

def decompose(a, n):
    # transform a to the format: a = LU
    for k in range(n - 1):
        for i in range(k + 1, n):
            factor = a[i][k] / a[k][k]
            a[i][k] = factor
            for j in range(k + 1, n):
                a[i][j] = a[i][j] - factor * a[k][j]
                
                
def substitute(a, n, b, x):
    # forward substitution: LD = b
    for i in range(1, n):
        _sum = b[i]
        for j in range(i):
            _sum -= a[i][j] * b[j]
        b[i] = _sum
    # back substitution: Ux = D
    x[n-1] = b[n-1] / a[n-1][n-1]
    for i in range(n - 2, -1, -1):
        _sum = 0
        for j in range(i + 1, n):
            _sum += a[i][j] * x[j]
        x[i] = (b[i] - _sum) / a[i][i]
    

2. Implement Calculation

In [3]:
a = np.array([[7, 2, -3], [2, 5, -3], [1, -1, -6]]).astype(float)

a_inverse = matrix_inverse(a.copy(), 3)
a_inverse

[[0.171875, -0.07812499999999999, -0.046875],
 [-0.04687499999999999, 0.20312499999999997, -0.078125],
 [0.03645833333333333, -0.04687499999999999, -0.16145833333333334]]

3. Check Answer

In [4]:
# 1. check the result
print(np.dot(a, a_inverse)) # should be identity matrix

[[1.00000000e+00 3.46944695e-17 2.77555756e-17]
 [4.16333634e-17 1.00000000e+00 2.77555756e-17]
 [2.77555756e-17 1.38777878e-17 1.00000000e+00]]


In [5]:
# 2. use the python library
p, l, u = linalg.lu(a)
p, l, u

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[ 1.        ,  0.        ,  0.        ],
        [ 0.28571429,  1.        ,  0.        ],
        [ 0.14285714, -0.29032258,  1.        ]]),
 array([[ 7.        ,  2.        , -3.        ],
        [ 0.        ,  4.42857143, -2.14285714],
        [ 0.        ,  0.        , -6.19354839]]))

In [6]:
a_inverse = linalg.inv(a)
a_inverse

array([[ 0.171875  , -0.078125  , -0.046875  ],
       [-0.046875  ,  0.203125  , -0.078125  ],
       [ 0.03645833, -0.046875  , -0.16145833]])

### Problem 11.12
Use the Gauss-Seidel method (a) without relaxation (b) with relaxation ($\lambda=0.95$) to solve the following system to a tolerance of $\epsilon_s=5\%$.  If necessary, rearrange the equations to achieve convergence.

$-3x_1 + 1x_2 + 15x_3 = 44$

$6x_1 - 2x_2 + x_3 = 5$

$5x_1 + 10x_2 + x_3 = 28$

### Answer

1. Define Function

In [7]:
def gseid(a, b, n, x, imax, es, _lambda):
    """
    This functions implements Gauss-Seidel method to solve linear equations
      using the pseudocode from figure 11.6 in the textbook
    """
    
    for i in range(n):
        dummy = a[i][i]
        for j in range(n):
            a[i][j] = a[i][j] / dummy
        b[i] = b[i] / dummy
    
    # first guess
    for i in range(n):
        _sum = b[i]
        for j in range(n):
            if i != j:
                _sum -= a[i][j] * x[j]
        x[i] = _sum

    # loops for iteration of more accurate guesses
    # stop until fall below the error criterion
    _iter = 1
    while True:    
        sentinel = 1
        for i in range(n):
            old = x[i]
            _sum = b[i]
            for j in range(n):
                if i != j:
                    _sum -= a[i][j] * x[j]
            x[i] = _lambda * _sum + (1 - _lambda) * old
            if sentinel == 1 and x[i] != 0:
                ea = abs((x[i] - old) / x[i]) * 100
                if ea > es:
                    sentinel = 0
           
        _iter += 1
        if sentinel == 1 or (_iter >= imax):
            break
            
    print('The result is {} with {} iterations'.format(x, _iter))

2. Implement Calculation

In [8]:
# original
#a = np.array([[-3, 1, 15], [6, -2, 1], [5, 10, 1]]).astype(float)
#b = np.array([44, 5, 28]).astype(float)

In [9]:
# ! need to rearrange the equations to achieve convergence
a = np.array([[6, -2, 1], [5, 10, 1], [-3, 1, 15]]).astype(float)
b = np.array([5, 28, 44]).astype(float)

In [10]:
# (a) if lambda is 1, the result is unmodified
x = np.array([0, 0, 0]).astype(float) # initial guesses
gseid(a.copy(), b.copy(), 3, x, 100, 5, 1)

The result is [1.00430445 1.99843924 3.00096494] with 4 iterations


In [11]:
# (b) with relaxation (𝜆 = 0.95)
x = np.array([0, 0, 0]).astype(float) # initial guesses
gseid(a.copy(), b.copy(), 3, x, 100, 5, 0.95)

The result is [0.99966339 2.00019778 2.99990359] with 4 iterations


3. Check Answer

In [12]:
# use the python library
x = np.linalg.solve(a, b)
x

array([1., 2., 3.])